In [105]:
import pandas as pd
import numpy as np
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [106]:
!pip install lexrank

In [107]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

In [108]:
sample_df1 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Lounge_Air/Sent Codes/Review/Scraped Review Data/Asia/India.xlsx',sheet_name = 1)
sample_df1 = sample_df1.dropna()
#sample_df1.tail()

In [109]:
sample_df = sample_df1[ sample_df1['Info1'] == 'American Express Lounge']

In [110]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [111]:
sia = SentimentIntensityAnalyzer()

In [112]:
"""sample_df['neg'] = sample_df['Review'].apply(lambda x:sia.polarity_scores(x)['neg'])
sample_df['neu'] = sample_df['Review'].apply(lambda x:sia.polarity_scores(x)['neu'])
sample_df['pos'] = sample_df['Review'].apply(lambda x:sia.polarity_scores(x)['pos'])"""
sample_df['Review Score'] = sample_df['Review'].apply(lambda x:sia.polarity_scores(x)['compound'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [113]:
sample_df.tail(1)

,Url,Info1,Info2,Reviewer,Oldness,Review,Stars,Country,Review Score
26912,"https://www.google.com/maps/place/American+Express+Lounge/@19.1009098,72.8575688,15z/data=!4m8!1m2!2m1!1slounges+near+Chhatrapati+Shivaji+International+Airport!3m4!1s0x3be7c9f7aa68d8ad:0x89dbd8b27f8fd0ad!8m2!3d19.0960952!4d72.8739814",American Express Lounge,Mumbai,Amit Satra,8 months ago,(Translated by Google) No A La carte menu\n\n(Original)\nNo A La carte menu,3 stars,India,-0.5267


In [114]:
def score_analyzer(score):
  if(score > 0.3):
    return 'Positive'
  elif(score< -0.3):
    return 'Negetive'
  else:
    return "Neutral"

In [115]:
sample_df['Review Category'] = sample_df['Review Score'].apply(score_analyzer)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [116]:
sample_df['Review Category'].value_counts()

Positive    626
Neutral     172
Negetive     38
Name: Review Category, dtype: int64

In [117]:
#sample_df.to_excel('sample.xlsx',sheet_name='Output',index=False)

In [118]:
#sample_df.drop(["Url"], axis=1,inplace = True)

In [119]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from lexrank import STOPWORDS, LexRank

In [120]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [121]:
new_stopwords = set(stopwords.words('english')) - {'not', 'no','nor','very','ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"}
print(new_stopwords)

{'is', 'both', 'too', 'under', 'out', 'some', 'd', 'his', 'than', 'm', 'only', 'themselves', 'as', 'where', 'by', 'she', 'while', "it's", 'and', 'to', 'i', 'its', 'the', 'we', 'don', 'our', 'being', 'down', 'been', 'at', 'this', 'me', 'during', "you'd", 'up', 'with', 'o', 'against', 'from', 'which', 'so', 'further', 'ours', 'your', "that'll", 'whom', 'here', 'can', 'how', 've', 'above', 'because', 'of', 'are', 'own', 'between', 'theirs', 'hers', 'yourself', 'then', 'when', 'he', 'there', 'off', "don't", 'again', 'were', 'until', 'about', 'below', 'same', 'all', 'yours', 'them', 'll', 'will', 'does', 're', 'him', 'was', 'had', "you're", 'doing', 'any', 's', 'himself', "you've", 'should', 'myself', 'my', 'y', 'her', "she's", 'have', 'just', 'who', 'it', 'these', 'did', 'ourselves', 'has', 'those', 'what', 'that', 'over', 'having', 'but', 'before', 'after', 'through', "you'll", 'now', 'their', 'into', 'they', "should've", 'on', 'do', 'for', 'more', 'most', 'once', 'itself', 'why', 'am', '

In [122]:
  lxr = LexRank(sample_df1['Review'], stopwords=new_stopwords)

In [123]:
r_list = []
r_sum = []
for texto in sample_df['Review']:
  text = sent_tokenize(texto)
  corpus = []
  for review_t in text:
    review = re.sub('[^a-zA-Z]', ' ', review_t) 
    review = review.lower() 
    review = review.split()

    ps = PorterStemmer()  
    #ps.stem(word)
    review = [word for word in review  if not word in set(new_stopwords)]  
    review = ' '.join(review) 
    corpus.append(review) 
  r_list.append(corpus)
  summary = lxr.get_summary(text, summary_size=2, threshold=.1)
  r_sum.append(summary[0])
sample_df['Phrases'] = r_list
sample_df['Summery'] = r_sum

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [124]:
sample_df

,Url,Info1,Info2,Reviewer,Oldness,Review,Stars,Country,Review Score,Review Category,Phrases,Summery
15452,"https://www.google.com/maps/place/American+Express+Lounge/@28.6153843,77.0352839,12z/data=!4m8!1m2!2m1!1sairport+lounges+near+Delhi!3m4!1s0x390d1b753079731b:0xb4e89c16ec32c2!8m2!3d28.5529492!4d77.084307",American Express Lounge,Delhi,vivek sethi,3 months ago,"Good ambience , Premium feeling , variety of eating options on Ala Carter. Beers and wine. You will feel the difference from any normal Lounge.",4 stars,India,0.5267,Positive,"[good ambience premium feeling variety eating options ala carter, beers wine, feel difference normal lounge]",You will feel the difference from any normal Lounge.
15453,"https://www.google.com/maps/place/American+Express+Lounge/@28.6153843,77.0352839,12z/data=!4m8!1m2!2m1!1sairport+lounges+near+Delhi!3m4!1s0x390d1b753079731b:0xb4e89c16ec32c2!8m2!3d28.5529492!4d77.084307",American Express Lounge,Delhi,Imran Khan,4 months ago,Nice & cozy lounge for AMEX card holders. Very comprehensive menu items. Better than the Plaza premium lounges at IGI airport.\nAMEX card holders should definitely consume there complementary lounge access.,5 stars,India,0.8655,Positive,"[nice cozy lounge amex card holders, very comprehensive menu items, better plaza premium lounges igi airport, amex card holders definitely consume complementary lounge access]",AMEX card holders should definitely consume there complementary lounge access.
15454,"https://www.google.com/maps/place/American+Express+Lounge/@28.6153843,77.0352839,12z/data=!4m8!1m2!2m1!1sairport+lounges+near+Delhi!3m4!1s0x390d1b753079731b:0xb4e89c16ec32c2!8m2!3d28.5529492!4d77.084307",American Express Lounge,Delhi,ashish sehgal,2 months ago,"Amazing place. I never thought the kind of food menu and drinks available, they were simply amazing. Hats off to AMEX team. Food was amazing. Very beautifully designed lounge.",5 stars,India,0.9468,Positive,"[amazing place, never thought kind food menu drinks available simply amazing, hats amex team, food amazing, very beautifully designed lounge]",Very beautifully designed lounge.
15455,"https://www.google.com/maps/place/American+Express+Lounge/@28.6153843,77.0352839,12z/data=!4m8!1m2!2m1!1sairport+lounges+near+Delhi!3m4!1s0x390d1b753079731b:0xb4e89c16ec32c2!8m2!3d28.5529492!4d77.084307",American Express Lounge,Delhi,Aayush Kumar Lal Das,2 months ago,"The lounge is good compared to other lounges, the lounge isn't always full and quiet with friendly helpful staff always ready to help. the food variety could be improved. Other than that the lounge is very good.",4 stars,India,0.9590,Positive,"[lounge good compared lounges lounge isn always full quiet friendly helpful staff always ready help, food variety could improved, lounge very good]",Other than that the lounge is very good.
15456,"https://www.google.com/maps/place/American+Express+Lounge/@28.6153843,77.0352839,12z/data=!4m8!1m2!2m1!1sairport+lounges+near+Delhi!3m4!1s0x390d1b753079731b:0xb4e89c16ec32c2!8m2!3d28.5529492!4d77.084307",American Express Lounge,Delhi,Rishikesh Nighot,6 months ago,Good collection of books to buy on the go. Would recommend this shop for anyone who is trying to spend some time shopping and going through some good titles. The store offers other things to buy too which is good.,4 stars,India,0.8807,Positive,"[good collection books buy go, would recommend shop anyone trying spend time shopping going good titles, store offers things buy good]",The store offers other things to buy too which is good.
15457,"https://www.google.com/maps/place/American+Express+Lounge/@28.6153843,77.0352839,12z/data=!4m8!1m2!2m1!1sairport+lounges+near+Delhi!3m4!1s0x390d1b753079731b:0xb4e89c16ec32c2!8m2!3d28.5529492!4d77.084307",American Express Lounge,Delhi,KESAVAN NAMBOOTHIRI.N,2 months ago,A very good rest place with Good Ambience variety of stores & food Joints .Really forget the time,5 stars,India,0.5994,Positive,[very good rest place good ambience variety stores food joi

In [125]:
"""
text = sample_df['Review'][0]
text = sent_tokenize(text)
corpus = []
for review_t in text:
 review = re.sub('[^a-zA-Z]', ' ', review_t) 
 review = review.lower() 
 review = review.split()

 ps = PorterStemmer()  
 #ps.stem(word)
 review = [word for word in review  if not word in set(stopwords.words('english'))]  
 review = ' '.join(review) 
 corpus.append(review) 
corpus
"""

"\ntext = sample_df['Review'][0]\ntext = sent_tokenize(text)\ncorpus = []\nfor review_t in text:\n review = re.sub('[^a-zA-Z]', ' ', review_t) \n review = review.lower() \n review = review.split()\n\n ps = PorterStemmer()  \n #ps.stem(word)\n review = [word for word in review  if not word in set(stopwords.words('english'))]  \n review = ' '.join(review) \n corpus.append(review) \ncorpus\n"

In [126]:
sample_df.to_excel('sample.xlsx')